In [2]:
import keras
import numpy as np
import os

from keras import backend as K
from keras.layers import Input, Conv2D,GlobalAveragePooling2D, Dense, BatchNormalization, Activation, MaxPooling2D
from keras.models import Model
from keras.layers import DepthwiseConv2D

from keras import optimizers,regularizers
from tensorflow_model_optimization.quantization.keras import vitis_quantize
from keras.initializers import he_normal
from keras.callbacks import LearningRateScheduler, TensorBoard, ModelCheckpoint
#from keras_flops import get_flops

def depthwise_separable(x,params):
    # f1/f2 filter size, s1 stride of conv
    (s1,f2) = params
    x = DepthwiseConv2D((3,3),strides=(s1[0],s1[0]), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(int(f2[0]), (1,1), strides=(1,1), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x


def MobileNet(img_input,shallow=False, classes=10):
    """Instantiates the MobileNet.Network has two hyper-parameters
        which are the width of network (controlled by alpha)
        and input size.
        # Arguments
            alpha: optional parameter of the network to change the 
                width of model.
            shallow: optional parameter for making network smaller.
            classes: optional number of classes to classify images
                into.
    """
    x = Conv2D(int(32), (4,4), strides=(2,2), padding='same')(img_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = depthwise_separable(x,params=[(1,),(64,)])
    x = depthwise_separable(x,params=[(2,),(128,)])
    x = depthwise_separable(x,params=[(1,),(128,)])
    x = depthwise_separable(x,params=[(2,),(256,)])
    x = depthwise_separable(x,params=[(1,),(256,)])
    x = depthwise_separable(x,params=[(2,),(512,)])
    
    if not shallow:
        for _ in range(5):
            x = depthwise_separable(x,params=[(1,),(512,)])
            
    x = depthwise_separable(x,params=[(2,),(1024,)])
    x = depthwise_separable(x,params=[(2,),(1024,)])

    x = GlobalAveragePooling2D()(x)
    out = Dense(classes, activation='softmax')(x)
    return out

shape_range =range(192,512,32)
for shape_x in shape_range:
    shape_y = shape_x
    X_train=np.random.randint(0,255,size=[60,shape_x,shape_y,1])
    Y_train=np.random.randint(0,10,size=[60,])
    X_test=np.random.randint(0,255,size=[60,shape_x,shape_y,1])
    Y_test=np.random.randint(0,10,size=[60,])
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255.0
    X_test /= 255.0
    img_input=Input(shape=(shape_x,shape_y,1))
    output = MobileNet(img_input)
    model=Model(img_input,output)
    model.summary()

    #FLOPs = round(get_flops(model, batch_size=1)/10 ** 9,2)

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train, Y_train, epochs=1)
    model.get_weights()[0].dtype
    quantizer = vitis_quantize.VitisQuantizer(model)
    quantized_model = quantizer.quantize_model(calib_dataset = X_test[1:10], weight_bit=8, activation_bit=8)
    quantized_model.compile(loss='sparse_categorical_crossentropy', metrics=["accuracy"])
    quantized_model.save('tf2_mnist_classifier_quantized.h5')
    !vai_c_tensorflow2 \
    --model ./tf2_mnist_classifier_quantized.h5 \
    --arch ./arch.json \
    --output_dir ./xmodel_MobileNet \
    --net_name deploy
    os.rename("./xmodel_MobileNet/deploy.xmodel","./xmodel_MobileNet/MobileNet_{shape_x}_{shape_y}.xmodel".format(shape_x=shape_x,shape_y=shape_y))

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 192, 192, 1)]     0         
_________________________________________________________________
conv2d_140 (Conv2D)          (None, 96, 96, 32)        544       
_________________________________________________________________
batch_normalization_270 (Bat (None, 96, 96, 32)        128       
_________________________________________________________________
activation_270 (Activation)  (None, 96, 96, 32)        0         
_________________________________________________________________
depthwise_conv2d_130 (Depthw (None, 96, 96, 32)        320       
_________________________________________________________________
batch_normalization_271 (Bat (None, 96, 96, 32)        128       
_________________________________________________________________
activation_271 (Activation)  (None, 96, 96, 32)        0  

2/2 [==============================] - 3s 855ms/step - loss: 2.5219 - accuracy: 0.1167
[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 4s 401ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 9s 9s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./tf2_mnist_classifier_quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/tf2_mnis

FileNotFoundError: [Errno 2] No such file or directory: './xmodel_MobileNet/deploy.xmodel' -> './xmodel_MobileNet/MobileNet_192_192.xmodel'